In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import json

In [2]:
directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：1



,No,Part Number,Vehicle,Picture,Url,Json_Type,Json_Info,Json_Specification
0,29,NS-173,Nissan D21 (94)\nNissan Frontier (99-98)\nNiss...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Switch - Clutch Starter Safety""}","{""Brand"": ""STI"", ""Partdesc_En"": ""Clutch Starte...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."


In [4]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                                   29
 Part Number                                                      NS-173
 Vehicle               Nissan D21 (94)\nNissan Frontier (99-98)\nNiss...
 Picture                                                                
 Url                   https://www.standardbrand.com/en/ecatalog?part...
 Json_Type                       {"0": "Switch - Clutch Starter Safety"}
 Json_Info             {"Brand": "STI", "Partdesc_En": "Clutch Starte...
 Json_Specification    {"0": {"Warranty": "3 years/36,000 mile"}, "1"...
 Name: 0, dtype: object)

In [5]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request, url_check = f'''https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?partNum={crawler_series['Part Number']}&brand={json.loads(crawler_series['Json_Info'])['Brand']}&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960''', crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

resp.json()

1
2


[]

In [6]:
dict_src = {}
for i, dict_ in enumerate(resp.json()):
    dict_src[str(i)] = dict_['image_BG_Url'].strip()

dict_src

{}

In [7]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Part Number': crawler_series['Part Number'],
                          'Vehicle': crawler_series['Vehicle'],
                          'Picture': crawler_series['Picture'],
                          'Url': crawler_series['Url'],
                          'Json_Src': json.dumps(dict_src),
                          'Json_Type': crawler_series['Json_Type'],
                          'Json_Info': crawler_series['Json_Info'],
                          'Json_Specification': crawler_series['Json_Specification']})

list_dict_correct

[{'No': 29,
  'Part Number': 'NS-173',
  'Vehicle': 'Nissan D21 (94)\nNissan Frontier (99-98)\nNissan Pickup (97-95)',
  'Picture': '',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=NS173&type=p&search=s',
  'Json_Src': '{}',
  'Json_Type': '{"0": "Switch - Clutch Starter Safety"}',
  'Json_Info': '{"Brand": "STI", "Partdesc_En": "Clutch Starter Safety Switch", "Partlongdesc_En": "Clutch Starter Safety Switch", "Pop": "X", "Qtyeach": 1}',
  'Json_Specification': '{"0": {"Warranty": "3 years/36,000 mile"}, "1": {"Connector Quantity": "2"}, "2": {"Connector Gender": "Female; Female"}, "3": {"Terminal Quantity": "2"}, "4": {"Terminal Gender": "Female; Male"}, "5": {"Terminal Type": "Blade Terminals"}, "6": {"Color/Finish": "Black, Gray"}, "7": {"Finish": "Dichromate, Plastic"}}'}]

In [8]:
crawler_status = True

crawler_status

True

In [9]:
crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                                   29
 Part Number                                                      NS-173
 Vehicle               Nissan D21 (94)\nNissan Frontier (99-98)\nNiss...
 Picture                                                                
 Url                   https://www.standardbrand.com/en/ecatalog?part...
 Json_Type                       {"0": "Switch - Clutch Starter Safety"}
 Json_Info             {"Brand": "STI", "Partdesc_En": "Clutch Starte...
 Json_Specification    {"0": {"Warranty": "3 years/36,000 mile"}, "1"...
 Request_Url           https://ecatalog.smpcorp.com/V2/STD/api/image/...
 Check_Url             https://www.standardbrand.com/en/ecatalog?part...
 Name: 0, dtype: object]

In [10]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_src.xlsx', index=False)

df_correct

,No,Part Number,Vehicle,Picture,Url,Json_Src,Json_Type,Json_Info,Json_Specification
0,29,NS-173,Nissan D21 (94)\nNissan Frontier (99-98)\nNiss...,,https://www.standardbrand.com/en/ecatalog?part...,{},"{""0"": ""Switch - Clutch Starter Safety""}","{""Brand"": ""STI"", ""Partdesc_En"": ""Clutch Starte...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."


In [11]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_src_error.xlsx', index=False)

df_error

,No,Part Number,Vehicle,Picture,Url,Json_Type,Json_Info,Json_Specification,Request_Url,Check_Url
0,29,NS-173,Nissan D21 (94)\nNissan Frontier (99-98)\nNiss...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Switch - Clutch Starter Safety""}","{""Brand"": ""STI"", ""Partdesc_En"": ""Clutch Starte...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""...",https://ecatalog.smpcorp.com/V2/STD/api/image/...,https://www.standardbrand.com/en/ecatalog?part...
